In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 40.7 MB/s 
     |████████████████████████████████| 101 kB 12.1 MB/s 
     |████████████████████████████████| 596 kB 58.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


#construct dataset  
dataset구성
spam dataset을 하나씩 반환하며  
BertTokenzier를 통해 tokenziation후 input_ids, token_type_ids, attention mask를 tensor로 바꿔서 return  
또한 label이 spam일때는 1 ham일때는 0을 함께 return  

In [12]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer,BertTokenizer
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

#dataset load
data_path = "/content/drive/MyDrive/NLP_tutorial/4일차_배포/dataset/spam.csv"
data_df = pd.read_csv(data_path, encoding = "ISO-8859-1")
print(data_df)
print(data_df.columns)

#train test split
data_df = data_df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)
train_df = data_df.loc[:200,:].reset_index()
test_df = data_df.loc[4000:,:].reset_index()
print(train_df)
print(train_df.columns)

#tokenizer load
tokenizer=get_tokenizer("basic_english")
d = data_df.loc[0:3,"v2"]
print(d)
print(tokenizer(data_df.loc[0,"v2"]))

#build vocab dictionary
vocab = build_vocab_from_iterator(list(map(tokenizer,data_df.loc[:,"v2"])))
print(vocab(tokenizer(data_df.loc[0,"v2"])))

#huggingface tokenizer load
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
print(tokenizer(data_df.loc[0,"v2"], max_length=100, padding='max_length', truncation=True))

class myDataset(Dataset):
  def __init__(self, df, tokenizer, vocab) -> None:
      super().__init__()
      self.df = df
      self.tokenizer = tokenizer
      self.vocab = vocab

  def __len__(self):
      return len(self.df)
  
  def __getitem__(self, index):
      data = self.df.loc[index, "v2"]
      target = self.df.loc[index, "v1"]
      
      #data tokenization
      data = self.tokenizer(data, max_length=100,padding='max_length', truncation=True)

      #labeling
      if target == "ham":
          label = 0
      elif target == "spam":
          label = 1

      input_ids = torch.IntTensor(data["input_ids"])
      token_type_ids = torch.IntTensor(data["token_type_ids"])
      attention_mask = torch.IntTensor(data["attention_mask"])
      return input_ids, token_type_ids, attention_mask, label


train_dataset = myDataset(train_df, tokenizer, vocab)
test_dataset = myDataset(test_df, tokenizer, vocab)

batch_size = 3

train_dataloader = DataLoader(train_dataset, batch_size = batch_size)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size)

for i in train_dataset:
  print(i)
  break

for i in train_dataloader:
  print(i)
  break
  

        v1                                                 v2 Unnamed: 2  \
0      ham  Go until jurong point, crazy.. Available only ...        NaN   
1      ham                      Ok lar... Joking wif u oni...        NaN   
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3      ham  U dun say so early hor... U c already then say...        NaN   
4      ham  Nah I don't think he goes to usf, he lives aro...        NaN   
...    ...                                                ...        ...   
5567  spam  This is the 2nd time we have tried 2 contact u...        NaN   
5568   ham              Will Ì_ b going to esplanade fr home?        NaN   
5569   ham  Pity, * was in mood for that. So...any other s...        NaN   
5570   ham  The guy did some bitching but I acted like i'd...        NaN   
5571   ham                         Rofl. Its true to its name        NaN   

     Unnamed: 3 Unnamed: 4  
0           NaN        NaN  
1           NaN        NaN  


##Model
BertModel을 이용해서 구현  
BertModel의 cls token에 대한 output인 pooler_output에 linear를 통해 2차원 벡터로 변환  
BertModel: https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel

In [13]:
from torch import nn
from transformers import BertModel


class MyModel(nn.Module):
  def __init__(self) -> None:
      super().__init__()
      self.bert = BertModel.from_pretrained("bert-base-uncased")
      self.ln1 = nn.Linear(768,1)

  def forward(self,input_ids, token_type_ids, attention_mask):
    out = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    out = out["last_hidden_state"]
    answer = self.ln1(out)
    answer = self.bert(out)
    

    return answer

model = MyModel()

for i in train_dataloader:
  input_ids = i[0]
  token_type_ids = i[1]
  attention_mask = i[2]
  out = model(input_ids, token_type_ids, attention_mask)
  #print(out[0].size(),out[1].size())
  #print(i[3])
  break

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


##train
cross entropy loss와 Adam을 통해 train

In [14]:
from torch.optim import Adam

model = MyModel()
model.cuda()

optimizer = Adam(model.parameters(), lr=0.0001)
lf = nn.CrossEntropyLoss()

for e in range(30):
  print("\n\n epoch", e)
  model.train()
  train_acc = 0
  epoch_loss = 0

  for i in train_dataloader:
    optimizer.zero_grad()
    input_ids = i[0]
    input_ids = input_ids.cuda()
    token_type_ids = i[1]
    token_type_ids = token_type_ids.cuda()
    attention_mask = i[2]
    attention_mask = attention_mask.cuda()
    target = i[3]
    target = target.cuda()

    output = model(input_ids,token_type_ids,attention_mask)
    print(output.shape)
    print(target.shape)

    pred_label = torch.argmax(output, dim=-1)
    target = target.reshape(-1)
    print(target.shape)
    loss = lf(output,target)
    train_acc += sum(pred_label == target.reshape(-1))

    loss.backward()
    optimizer.step()

    epoch_loss += loss.item()

print(train_acc)
print("train loss", epoch_loss/len(train_dataloader))
print("train acc", train_acc/len(train_dataset))


    

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: ignored